<img src="https://news.illinois.edu/files/6367/543635/116641.jpg" alt="University of Illinois" width="250"/>

# PyTorch RNN for Energy Data#
By Richard Sowers
* <r-sowers@illinois.edu>
* <https://publish.illinois.edu/r-sowers/>

Copyright 2020 University of Illinois Board of Trustees. All Rights Reserved.

# Imports and Configurations

In [ ]:
import os
import numpy
import pandas
idx = pandas.IndexSlice
import time
import random
import matplotlib
#%matplotlib notebook
import matplotlib.pyplot as plt
import scipy.stats
#from pandas.plotting import autocorrelation_plot
import matplotlib.offsetbox as offsetbox
from matplotlib.ticker import StrMethodFormatter
from matplotlib.backends.backend_agg import FigureCanvasAgg
import graphviz

import imageio
import PIL

def saver(fname):
    plt.savefig(fname+".png",bbox_inches="tight")

def legend(pos="bottom",ncol=3):
    if pos=="bottom":
        plt.legend(bbox_to_anchor=(0.5,-0.2), loc='upper center',facecolor="lightgray",ncol=ncol)
    elif pos=="side":
        plt.legend(bbox_to_anchor=(1.1,0.5), loc='center left',facecolor="lightgray",ncol=1)

def textbox(txt,fname=None):
    plt.figure(figsize=(1,1))
    plt.gca().add_artist(offsetbox.AnchoredText("\n".join(txt), loc="center",prop=dict(size=30)))
    plt.axis('off')
    if fname is not None:
        saver(fname)
    plt.show()
    plt.close()

In [ ]:
import torch
import scipy

In [ ]:
#for some reason, this needs to be in a separate cell
params={
    "font.size":15,
    "lines.linewidth":5,
}
plt.rcParams.update(params)

In [ ]:
def getfile(location_pair,**kwargs): #tries to get local version and then defaults to google drive version
    (loc,gdrive)=location_pair
    try:
        out=pandas.read_csv(loc,**kwargs)
    except FileNotFoundError:
        print("local file not found; accessing Google Drive")
        loc = 'https://drive.google.com/uc?export=download&id='+gdrive.split('/')[-2]
        out=pandas.read_csv(loc,**kwargs)
    return out

In [ ]:
fname_actual=("Actual.csv","https://drive.google.com/file/d/1FfKANnGzXly62duW7hub4nXbnRJX6GsY/view?usp=sharing")
fname_dayahead=("DayAhead.csv","https://drive.google.com/file/d/1PDC6x4HnUmvUyJTS9GZozZZn3DV6X9R3/view?usp=sharing")

To understand PyTorch's RNN of <https://pytorch.org/docs/stable/generated/torch.nn.RNN.html>, let's implement some simple linear systems.  We will do this in an approximate way, reflecting the fact that RNN's typically have a nonlinearity.

# Data #

Let's read our energy data

In [ ]:
actual_raw=getfile(fname_actual)
actual_raw.head()

local file not found; accessing Google Drive


,Date,LMP,HUB
0,1/1/2021 10:00:00 PM,18.71,ARKANSAS.HUB
1,1/1/2021 10:00:00 PM,18.95,ILLINOIS.HUB
2,1/1/2021 10:00:00 PM,19.43,INDIANA.HUB
3,1/1/2021 10:00:00 PM,19.39,LOUISIANA.HUB
4,1/1/2021 10:00:00 PM,19.57,MICHIGAN.HUB


In [ ]:
actual=actual_raw.copy()
actual=actual.rename(columns={"LMP":"Actual Price"})
actual["Date"]=pandas.to_datetime(actual["Date"])
actual=actual.set_index(["Date","HUB"],append=False,drop=True)
actual.head()

Actual Price
Date                HUB                        
2021-01-01 22:00:00 ARKANSAS.HUB          18.71
                    ILLINOIS.HUB          18.95
                    INDIANA.HUB           19.43
                    LOUISIANA.HUB         19.39
                    MICHIGAN.HUB          19.57

In [ ]:
dayahead_raw=getfile(fname_dayahead)
dayahead_raw.head()

local file not found; accessing Google Drive


,Date,lmp,node
0,1/1/2021 12:00:00 AM,18.95,ARKANSAS.HUB
1,1/1/2021 12:00:00 AM,18.82,ILLINOIS.HUB
2,1/1/2021 12:00:00 AM,19.56,INDIANA.HUB
3,1/1/2021 12:00:00 AM,19.56,LOUISIANA.HUB
4,1/1/2021 12:00:00 AM,19.90,MICHIGAN.HUB


In [ ]:
dayahead=dayahead_raw.copy()
dayahead=dayahead.rename(columns={"lmp":"DayAhead Price","node":"HUB"})
dayahead["Date"]=pandas.to_datetime(dayahead["Date"])
dayahead=dayahead.set_index(["Date","HUB"],append=False,drop=True)
dayahead.head()

DayAhead Price
Date       HUB                          
2021-01-01 ARKANSAS.HUB            18.95
           ILLINOIS.HUB            18.82
           INDIANA.HUB             19.56
           LOUISIANA.HUB           19.56
           MICHIGAN.HUB            19.90

In [ ]:
joined=pandas.concat([actual,dayahead],axis="columns",join="inner")
illinois=joined.query("HUB=='ILLINOIS.HUB'").sort_index(axis="index").reset_index("HUB",drop=True).copy()
illinois.head()

,Actual Price,DayAhead Price
Date,,
2021-01-01 22:00:00,18.95,20.16
2021-01-02 22:00:00,20.19,20.94
2021-01-02 23:00:00,19.68,20.06
2021-01-03 00:00:00,19.51,19.67
2021-01-03 01:00:00,18.03,19.54


In [ ]:
RNN_data=illinois.rename(columns={"DayAhead Price":"X","Actual Price":"Y"})/100
RNN_data["t"]=range(len(RNN_data))
RNN_data=RNN_data.set_index("t",append=False,drop=True)
RNN_data=RNN_data[["X","Y"]]
RNN_data.head(3)

,X,Y
t,,
0,0.2016,0.1895
1,0.2094,0.2019
2,0.2006,0.1968


# Jordan Network #

In [ ]:
class Jordan(torch.nn.Module):
  def __init__(self, inputSize=2,hiddenSize=1,outputSize=1,SEED=0):
    super().__init__()
    if SEED is not None:
          torch.manual_seed(SEED)
    self.tanh=torch.nn.Tanh()
    self.linear_1 = torch.nn.Linear(inputSize,hiddenSize)
    self.linear_2 = torch.nn.Linear(hiddenSize,outputSize)
    self.ReLU=torch.nn.ReLU()
    if torch.cuda.is_available():
        "converting to cuda"
        self = self.cuda()

  def forward(self,input):
    out=self.linear_1(input)
    out=self.tanh(out)
    out=self.linear_2(out)
    out=self.ReLU(out)
    return out

Loss = torch.nn.MSELoss()

In [ ]:
Jordan_data=RNN_data.copy()
Jordan_data["lagged Y"]=Jordan_data["Y"].shift()
Jordan_data=Jordan_data[["X","lagged Y","Y"]]
Jordan_data=Jordan_data.dropna(axis='index')
Jordan_data.columns=pandas.MultiIndex.from_tuples(zip(["Input","Input","Output"],Jordan_data.columns))
Jordan_data.head()

Input           Output
        X lagged Y       Y
t                         
1  0.2094   0.1895  0.2019
2  0.2006   0.2019  0.1968
3  0.1967   0.1968  0.1951
4  0.1954   0.1951  0.1803
5  0.1943   0.1803  0.1812

In [ ]:
Jordan_features=torch.from_numpy(Jordan_data.loc[:,idx["Input",:]].values.astype(numpy.float32))
Jordan_labels=torch.from_numpy(Jordan_data.loc[:,idx["Output",:]].values.astype(numpy.float32))

In [ ]:
Jordan_model=Jordan(hiddenSize=2)

In [ ]:
optimizer = torch.optim.Adam(Jordan_model.parameters())
losses=[]
MAX_iter=100
for ctr in range(MAX_iter):

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = Jordan_model(Jordan_features)

    # get loss for the predicted output
    lossvalue = Loss(outputs, Jordan_labels)
    losses.append(lossvalue)

    # get gradients w.r.t to parameters
    lossvalue.backward()
    #print(model.linear.weight.grad.item(),model.linear.bias.grad.item())

    # update parameters
    optimizer.step()
    if ctr%int(MAX_iter/10)==0: #print out data for 10 intermediate steps
      print("iteration {}: loss={:.5f}".format(ctr, lossvalue.item()))

print("final loss={:.5f}".format(lossvalue.item()))

iteration 0: loss=0.21165
iteration 10: loss=0.21165
iteration 20: loss=0.21165
iteration 30: loss=0.21165
iteration 40: loss=0.21165
iteration 50: loss=0.21165
iteration 60: loss=0.21165
iteration 70: loss=0.21165
iteration 80: loss=0.21165
iteration 90: loss=0.21165
final loss=0.21165


# Elman network

## check explicit calculations ##

In [ ]:
class Elman(torch.nn.Module):
    def __init__(self, inputSize=1, hiddenSize=1,outputSize=1,numlayers=1,batchsize=1,SEED=0):
        #inputSize-dimensional inputs
        #hiddenSize-dimensional latent (plant) process
        #outputSize-dimensional output
        super().__init__() #run init of torch.nn.Module
        if SEED is not None:
          torch.manual_seed(SEED)
        self.rnn = torch.nn.RNN(inputSize,hiddenSize,numlayers,batch_first=True)
        # h_init defaults to zero
        self.linear = torch.nn.Linear(hiddenSize,outputSize)
        self.tanh=torch.nn.Tanh()
        self.ReLU=torch.nn.ReLU()
        if torch.cuda.is_available():
          "converting to cuda"
          self = self.cuda()

    def set_parameters(self,parameters):
      (w_ih,w_hh,b_h,w_o,b_o)=parameters
      self.linear.weight.data=torch.Tensor([[w_o]])
      self.linear.bias.data=torch.Tensor([b_o])
      self.rnn.weight_ih_l0.data=torch.Tensor([[w_ih]])
      self.rnn.bias_ih_l0.data=torch.Tensor([[b_h]])
      self.rnn.weight_hh_l0.data=torch.Tensor([[w_hh]])
      self.rnn.bias_hh_l0.data=torch.Tensor([[0]])

    def initialize(self,h_init):
      self.h_init=torch.from_numpy(numpy.array(h_init).astype(numpy.float32).reshape(1,1,1))

    def forward(self, inputs):
        out,_=self.rnn(inputs,self.h_init)
        self.rnn_out=out
        out=self.ReLU(out)
        return out

Loss = torch.nn.MSELoss()

In [ ]:
L=3 #sequence length
Elman_check_data=RNN_data.head(L).copy()
Elman_check_features=torch.from_numpy(Elman_check_data.loc[:,"X"].values.astype(numpy.float32).reshape(1,-1,1))
Elman_check_labels=torch.from_numpy(Elman_check_data.loc[:,"Y"].values.astype(numpy.float32).reshape(1,-1,1))

In [ ]:
parameters=(0.2,-0.25,0.3,0.1,-0.15)
#(w_o,b_o,w_ih,w_hh,b_h)

In [ ]:
Elman_check_model=Elman()
Elman_check_model.set_parameters(parameters)
Elman_check_model.initialize(0.1)

In [ ]:
Elman_check_output=Elman_check_model(Elman_check_features)
Elman_check_lossvalue=Loss(Elman_check_output,Elman_check_labels)

In [ ]:
print("SHOULD AGREE WITH EXPLICIT CALCULATIONS")
print("hidden layer={:s}".format(str(Elman_check_model.rnn_out.data.squeeze().numpy())))
print("model output={:s}".format(str(Elman_check_output.data.squeeze().numpy())))
print("loss={:.3f}".format(Elman_check_lossvalue.item()))

SHOULD AGREE WITH EXPLICIT CALCULATIONS
hidden layer=[0.30526915 0.25949115 0.2685006 ]
model output=[0.30526915 0.25949115 0.2685006 ]
loss=0.007


In [ ]:
#Elman_check_model.zero_grad()
#Elman_check_lossvalue.backward(retain_graph=True)
#print("SHOULD AGREE WITH EXPLICIT CALCULATIONS")
#print("partial of loss with respect to w_o={:.3f}".format(Elman_check_model.linear.weight.grad.item()))
#print("partial of loss with respect to b_o={:.3f}".format(Elman_check_model.linear.bias.grad.item()))
#print("partial of loss with respect to w_hh={:.4f}".format(Elman_check_model.rnn.weight_hh_l0.grad.item()))
#print("partial of loss with respect to w_ih={:.4f}".format(Elman_check_model.rnn.weight_ih_l0.grad.item()))
#print("partial of loss with respect to b_hh={:.4f}".format(Elman_check_model.rnn.bias_ih_l0.grad.item()))
#print("partial of loss with respect to b_ih={:.4f}".format(Elman_check_model.rnn.bias_ih_l0.grad.item()))


## Elman network for full dataset ##

In [ ]:
Elman_data=RNN_data.copy()
Elman_features=torch.from_numpy(Elman_data.loc[:,"X"].values.astype(numpy.float32).reshape(1,-1,1))
Elman_labels=torch.from_numpy(Elman_data.loc[:,"Y"].values.astype(numpy.float32).reshape(1,-1,1))
h_init=0

In [ ]:
Elman_model=Elman()
Elman_model.initialize(h_init)

In [ ]:
optimizer = torch.optim.Adam(Elman_model.parameters())
losses=[]
MAX_iter=100
for ctr in range(MAX_iter):

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = Elman_model(Elman_features)

    # get loss for the predicted output
    lossvalue = Loss(outputs, Elman_labels)
    losses.append(lossvalue)

    # get gradients w.r.t to parameters
    lossvalue.backward()
    #print(model.linear.weight.grad.item(),model.linear.bias.grad.item())

    # update parameters
    optimizer.step()
    if ctr%int(MAX_iter/10)==0: #print out data for 10 intermediate steps
      print("iteration {}: loss={:.5f}".format(ctr, lossvalue.item()))

print("final loss={:.5f}".format(lossvalue.item()))

iteration 0: loss=0.21158
iteration 10: loss=0.21158
iteration 20: loss=0.21158
iteration 30: loss=0.21158
iteration 40: loss=0.21158
iteration 50: loss=0.21158
iteration 60: loss=0.21158
iteration 70: loss=0.21158
iteration 80: loss=0.21158
iteration 90: loss=0.21158
final loss=0.21158
